# YAP/TAZ quantification
## 05/31/23

### Segmentation using Cellpose. Segment nucleus and cytoplasm. Quantify YAP intensity.

### Author: Andy D. Tran, CCR Microscopy Core, LCBG, CCR, NCI

In [1]:
# Import libraries-------------------------------------------------------------------------------------------------

import os 
import numpy as np
import pandas as pd 
import napari 
import re 

from scipy import ndimage as nd 
from aicsimageio import AICSImage
from tifffile import imread, imwrite 
from skimage.measure import label, regionprops
from cellpose import models, io 
from tqdm import tqdm

In [6]:
# Define paths------------------------------------------------------------------------------------------------------

base_path = '/Volumes/LECIMAGE/Analysis/[NCI] [LCBG] Lalage Wakefield/Binwu Tang/images/new_Ab_Yap/10_23'
output_path = '/Volumes/LECIMAGE/Analysis/[NCI] [LCBG] Lalage Wakefield/Binwu Tang/output/New_Ab_Yap/10_23'

nuc_model = models.CellposeModel(pretrained_model = '/Volumes/LECIMAGE/Analysis/[NCI] [LCBG] Lalage Wakefield/Binwu Tang/training/yap_taz_nuclei/models/tangb_yaptaz_nuc_01')
cyto_model = models.CellposeModel(gpu = False, model_type = 'cyto3')

In [7]:
# Define functions--------------------------------------------------------------------------------------------------

def roi_quant(lbl_img, yap_img, sore6_img):
    df = pd.DataFrame() 
    
    cell_id = []
    area = [] 
    yap_intensity = [] 
    sore6_intensity = []
    
    roi_props = regionprops(lbl_img, intensity_image = yap_img)
    sore_props = regionprops(lbl_img, intensity_image = sore6_img)
    
    for roi in tqdm(range(len(roi_props))):
        cell_id.append(roi_props[roi].label)
        area.append(roi_props[roi].area)
        yap_intensity.append(roi_props[roi].mean_intensity)
        sore6_intensity.append(sore_props[roi].mean_intensity)
        
    df['cell_id'] = cell_id
    df['area'] = area
    df['yap_intensity'] = yap_intensity
    df['sore6_intensity'] = sore6_intensity
    
    return df

def model_apply(img, diam, model, seg):
    if seg == 'nuclei':
        masks, flows, styles = model.eval(img, diameter = diam, channels = [0, 0])
        masks = np.uint16(masks)
    else:
        masks, flows, styles = model.eval(img, diameter = diam, channels = [2, 1])
        masks = np.uint16(masks)
    
    return masks

### Main function

In [8]:
# Get image list----------------------------------------------------------------------------------------------------

tmp_list = os.listdir(base_path)
img_list = [] 

for tmp in tmp_list:
    if re.search('.nd2', tmp):
        img_list.append(tmp)
        
print(img_list)
print(len(img_list))

['JM115A s6 100k 40x allab007.nd2', 'JM115A s6 12.5k 40xallab013.nd2', 'JM115A s6 12.5k 40xallab017.nd2', 'JM115A s6 12.5k 40xallab012.nd2', 'JM115A s6 12.5k 40xallab002.nd2', 'JM115A s6 12.5k 40xallab009.nd2', 'JM115A s6 12.5k 40xallab005.nd2', 'JM115A s6 12.5k 40xallab019.nd2', 'JM115A s6 100k 40x allab009.nd2', 'JM115A s6 12.5k 40xallab003.nd2', 'JM115Amcmv 40x all ab2.nd2', 'JM115A s6 100k 40x allab.nd2', 'JM115A s6 100k 40x allab004.nd2', 'JM115A s6 100k 40x allab002.nd2', 'JM115A s650k 40x no ab001.nd2', 'JM115A s6 100k 40x allab008.nd2', 'JM115A s6 12.5k 40xallab011.nd2', 'JM115A s6 100k 40x allab001.nd2', 'JM115A s6 12.5k 40xallab020.nd2', 'JM115A s6 12.5k 40xallab010.nd2', 'JM115A s6 100k 40x allab005.nd2', 'JM115A s6 12.5k 40xallab018.nd2', 'JM115A s650k 40x mcheery001.nd2', 'JM115A s650k 40x Yap 001.nd2', 'JM115A s6 100k 40x allab003.nd2', 'JM115A s6 12.5k 40xallab016.nd2', 'JM115A s6 12.5k 40xallab015.nd2', 'JM115Amcmv 40x allab002.nd2', 'JM115A s650k 40x mcheery.nd2', 'JM1

In [9]:
# Loop through images-----------------------------------------------------------------------------------------------

for img_name in img_list:
    print(img_name)
    
    img_path = os.path.join(base_path, img_name)
    tmp_img = AICSImage(img_path)
    print(tmp_img.shape)
    img = tmp_img.data
    
    dapi_img = img[0, 0, 0, :, :]
    gfp_img = img[0, 1, 0, :, :]
    sore6_img = img[0, 2, 0, :, :]
    yap_img = img[0, 3, 0, :, :] 
    
    nuc_label = model_apply(dapi_img, 50, nuc_model, 'nuclei')
    
    cell_tmp = np.add(gfp_img, dapi_img)
    cell_tmp = np.where(nuc_label > 0, cell_tmp, gfp_img * 5)
    cell_tmp = np.expand_dims(cell_tmp, axis = 0)
    dapi_tmp = np.expand_dims(dapi_img, axis = 0)
    cell_tmp = np.concatenate((dapi_tmp, cell_tmp), axis = 0)
       
    cell_label = model_apply(cell_tmp, 100, cyto_model, 'cyto3')
    nuc_label = np.where(nuc_label > 0, cell_label, 0)
    cyto_label = np.where(nuc_label > 0, 0, cell_label)
    
    nuc_df = roi_quant(nuc_label, yap_img, sore6_img)
    nuc_df = nuc_df.rename(columns = {'area' : 'nuclear_area', 'yap_intensity' : 'nuclear_yap_intensity', 'sore6_intensity' : 'nuclear_sore6_intensity'})

    cell_df = roi_quant(cell_label, yap_img, sore6_img)
    cell_df = cell_df.rename(columns = {'area' : 'cell_area', 'intensity' : 'cell_intensity', 'sore6_intensity' : 'cell_sore6_intensity'})
    
    cyto_df = roi_quant(cyto_label, yap_img, sore6_img)
    cyto_df = cyto_df.rename(columns = {'area' : 'cyto_area', 'intensity' : 'cyto_intensity', 'sore6_intensity' : 'cyto_sore6_intensity'})

    df = pd.merge(nuc_df, cell_df, how = 'left', on = ['cell_id'])
    df = pd.merge(df, cyto_df, how = 'left', on = ['cell_id'])
    
    dapi_path = os.path.join(output_path, img_name.replace('.nd2', '_dapi.tif'))
    imwrite(dapi_path, dapi_img)
    
    gfp_path = os.path.join(output_path, img_name.replace('.nd2', '_gfp.tif'))
    imwrite(gfp_path, gfp_img)
    
    sore6_path = os.path.join(output_path, img_name.replace('.nd2', '_sore6.tif'))
    imwrite(sore6_path, sore6_img)
    
    yap_path = os.path.join(output_path, img_name.replace('.nd2', '_yap.tif'))
    imwrite(yap_path, yap_img)
    
    nuc_label_path = os.path.join(output_path, img_name.replace('.nd2', '_nuclei.tif'))
    nuc_label = np.uint16(nuc_label)
    imwrite(nuc_label_path, nuc_label)
    
    cell_label_path = os.path.join(output_path, img_name.replace('.nd2', '_cell.tif'))
    cyto_label = np.uint16(cyto_label)
    imwrite(cell_label_path, cell_label)
    
    cyto_label_path = os.path.join(output_path, img_name.replace('.nd2', '_cyto.tif'))
    imwrite(cyto_label_path, cyto_label)
    
    df_path = os.path.join(output_path, img_name.replace('.nd2', '_output.csv'))
    df.to_csv(df_path, header = True)

JM115A s6 100k 40x allab007.nd2
(1, 4, 1, 2200, 2200)


100%|███████████████████████████████████████| 151/151 [00:00<00:00, 7330.83it/s]


JM115A s6 12.5k 40xallab013.nd2
(1, 4, 1, 2200, 2200)


100%|█████████████████████████████████████████| 64/64 [00:00<00:00, 3890.14it/s]


JM115A s6 12.5k 40xallab017.nd2
(1, 4, 1, 2200, 2200)


...

100%|███████████████████████████████████████| 102/102 [00:00<00:00, 3951.37it/s]


JM115Amcmv 40x all ab2001.nd2
(1, 4, 1, 2200, 2200)


100%|█████████████████████████████████████████| 60/60 [00:00<00:00, 5347.04it/s]


JM115A s6 12.5k 40xallab.nd2
(1, 4, 1, 2200, 2200)


100%|█████████████████████████████████████████| 79/79 [00:00<00:00, 2345.87it/s]


### Napari visualization

In [11]:
# Get image list----------------------------------------------------------------------------------------------------

tmp_list = os.listdir(output_path)
img_list = [] 

for tmp in tmp_list:
    if re.search('_dapi.tif', tmp):
        img_list.append(tmp.replace('_dapi.tif', ''))
        
print(img_list)
print(len(img_list))

['JM115Amcmv 40x allab', 'JM115Amcmv 40x allab002', 'JM115Amcmv 40x all ab2003', 'JM115A s6 12.5k 40xallab012', 'JM115A s650k 40x Yap ', 'JM115A s6 12.5k 40xallab014', 'JM115A s6 100k 40x allab', 'JM115A s6 100k 40x allab003', 'JM115A s6 12.5k 40xallab011', 'JM115A s6 100k 40x allab008', 'JM115A s6 12.5k 40xallab009', 'JM115A s650k 40x mcheery', 'JM115A s6 12.5k 40xallab006', 'JM115A s6 12.5k 40xallab001', 'JM115A s6 100k 40x allab005', 'JM115A s6 12.5k 40xallab', 'JM115A s6 100k 40x allab002', 'JM115A s6 12.5k 40xallab018', 'JM115A s6 12.5k 40xallab002', 'JM115A s6 12.5k 40xallab007', 'JM115A s650k 40x Yap 001', 'JM115Amcmv 40x all ab2001', 'JM115A s6 12.5k 40xallab004', 'JM115A s6 100k 40x allab006', 'JM115A s6 12.5k 40xallab005', 'JM115A s6 100k 40x allab009', 'JM115A s6 12.5k 40xallab020', 'JM115A s6 12.5k 40xallab013', 'JM115A s6 12.5k 40xallab010', 'JM115A s6 12.5k 40xallab008', 'JM115Amcmv 40x all ab2002', 'JM115A s650k 40x no ab', 'JM115A s6 100k 40x allab007', 'JM115A s6 100k 

In [17]:
img_name = img_list[2]
print(img_name)

dapi_path = os.path.join(output_path, img_name + '_dapi.tif')
dapi_img = imread(dapi_path)

gfp_path = os.path.join(output_path, img_name + '_gfp.tif')
gfp_img = imread(gfp_path)

sore6_path = os.path.join(output_path, img_name + '_sore6.tif')
sore6_img = imread(sore6_path)

yap_path = os.path.join(output_path, img_name + '_yap.tif')
yap_img = imread(yap_path)

nuc_label_path = os.path.join(output_path, img_name +  '_nuclei.tif')
nuc_label = imread(nuc_label_path)
    
cell_label_path = os.path.join(output_path, img_name +  '_cell.tif')
cell_label = imread(cell_label_path)
    
cyto_label_path = os.path.join(output_path, img_name + '_cyto.tif')
cyto_label = imread(cyto_label_path)

JM115Amcmv 40x all ab2003


In [18]:
viewer = napari.view_image(dapi_img, name = 'DAPI', colormap = 'blue', blending = 'additive')
gfp_layer = viewer.add_image(gfp_img, name = 'GFP', colormap = 'green', blending = 'additive')
sore6_layer = viewer.add_image(sore6_img, name = 'SORE6', colormap = 'red', blending = 'additive')
yap_layer = viewer.add_image(yap_img, name = 'YAP', colormap = 'gray', blending = 'additive')
nuc_label_layer = viewer.add_labels(nuc_label, name = 'Nuclear label')
cell_label_layer = viewer.add_labels(cell_label, name = 'Cell label')
cyto_label_layer = viewer.add_labels(cyto_label, name = 'Cytoplasmic label')

In [19]:
viewer.close()